In [1]:
from bs4 import BeautifulSoup
import urllib
import re

In [42]:
def getsoup (address):
    getRequest = urllib.request.Request(address, None, {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:54.0) Gecko/20100101 Firefox/54.0'})

    urlfile = urllib.request.urlopen(getRequest)
    htmlResult = urlfile.read(20000000)
    urlfile.close()
    
    return BeautifulSoup(htmlResult)
    
    
def search(query):
    addr = "http://www.bing.com/search?q=%s" % (urllib.parse.quote_plus(query)+ "&count=20")

    soup = getsoup(addr).find('div',id='b_content')

    [s.extract() for s in soup('span')]
    unwantedTags = ['cite']
    
    for tag in unwantedTags:
        for match in soup.findAll(tag):
            match.replaceWithChildren()

    results = soup.findAll('li')
    

    return results

def get_item_info(query):
    links = search(query)
    titles = []
    urls = []
    paragraphs = []
    for link in links :
        try:
            urls.append(str(link.find('h2').find('a')['href']).replace(" ", " "))
            titles.append(str(link.find('h2').text).replace(" ", " "))
        except:
            ""
        try:
            paragraphs.append(str(link.findAll('p')[0].text))
        except:
            paragraphs.append("")

    return titles, urls, paragraphs, links[0]

In [62]:
def getmenus():
    addr = "https://www.fastfoodmenuprices.com/all-restaurants/"

    soup = getsoup(addr)

    results = soup.findAll('ul', class_ = "menu")

    items = results[0].findAll('li')
    menus = []
    temp = {}
    for r in results:
        items = r.findAll('li')

        for i in items:
            menus.append({'name':i.find('a').text, 'url':i.find('a')['href']})
    
    return menus
            


In [63]:
menus = getmenus()
print(menus)


[{'name': '&pizza', 'url': 'https://www.fastfoodmenuprices.com/and-pizza-prices/'}, {'name': '99 Restaurant & Pub', 'url': 'https://www.fastfoodmenuprices.com/99-restaurant-pub-menu-prices/'}, {'name': 'Abuelo’s', 'url': 'https://www.fastfoodmenuprices.com/abuelos-menu-prices/'}, {'name': 'America’s Best Wings', 'url': 'https://www.fastfoodmenuprices.com/americas-best-wings-menu-prices/'}, {'name': 'American Deli’s', 'url': 'https://www.fastfoodmenuprices.com/american-delis-menu-prices/'}, {'name': 'Amy’s Drive Thru', 'url': 'https://www.fastfoodmenuprices.com/amys-drive-through-prices/'}, {'name': 'Andy’s Frozen Custard', 'url': 'https://www.fastfoodmenuprices.com/andys-frozen-custard-menu-prices/'}, {'name': 'Applebee’s', 'url': 'https://www.fastfoodmenuprices.com/applebees-prices/'}, {'name': 'Alinea', 'url': 'https://www.fastfoodmenuprices.com/alinea-menu-prices/'}, {'name': 'A&W', 'url': 'https://www.fastfoodmenuprices.com/aw-prices/'}, {'name': 'Arby’s', 'url': 'https://www.fastf

In [91]:
def getmenu(addr):
    soup = getsoup(addr)
    
    isweird = False

    names = soup.findAll('td', class_="column-1")
    
    prices = soup.findAll('td', class_="column-3")
    
    if len(prices)==0:
        
        isweird = True
        prices = soup.findAll('td', class_="column-2")


    menu = []

    for i in range(0,len(prices)):
        if names[i].findAll('h2') == []:
            menu.append({'item':names[i].text, 'price':prices[i].text})
        elif names[i].findAll('b') is not []:
            try:
                menu.append({'item':names[i].find('b').text, 'price':prices[i].find('span') .text})
            except:
                ""
    return menu

men = getmenu(menus[3]['url'])

In [92]:
for m in men:
    print(m)

{'item': 'Garden Salad', 'price': '$6.49'}
{'item': 'Crispy Chicken Salad', 'price': '$6.49'}
{'item': 'Chef Salad', 'price': '$6.99'}
{'item': 'Tilapia Fish Salad\nFried or grilled', 'price': '$6.49'}
{'item': 'Cheese Steak Salad', 'price': '$6.49'}
{'item': 'Grilled Chicken Salad', 'price': '$7.49'}
{'item': 'Catfish Salad\nFried or grilled', 'price': '$7.99'}
{'item': 'Shrimp Salad', 'price': '$3.49 - $139.99'}
{'item': 'Buffalo Wings\nBone-in, price depends on the number of wings', 'price': '$3.99 - $64.99'}
{'item': 'Buffalo Wing Combo\nWith fries & 20 oz. drink', 'price': '$9.49'}
{'item': 'Boneless Wings\nPrice depends on the number of wings.', 'price': '$5.99'}
{'item': 'Boneless Wings Combo\nComes with fries & 20 oz. drink', 'price': 'Sm - $5.39\n$7.99'}
{'item': "Boneless Kid's Meal\n5 pieces boneless wings, fries, and drink", 'price': 'Sm - $5.39\n$7.99'}
{'item': 'Steak Wrap', 'price': 'Sm - $5.99\n$8.89'}
{'item': 'Caesar Wrap', 'price': 'Sm - $5.39\n$7.99'}
{'item': 'Buff

In [98]:
addr = "https://www.allrecipes.com/search/results/?search=veggie+burger"

soup = getsoup(addr)

items = soup.findAll('div', role="listitem")

In [110]:
alternatives = []

for i in range(0, len(items)):
    tmp = {}
    tmp['name'] = items[i].find('img')['alt']
    tmp['img'] = items[i].find('img')['src']
    try:
        tmp['details'] = items[i].find("a", title = items[i].find('img')['alt'])['href']
    except:
        print("something wrong")
    alternatives.append(tmp)

something wrong


In [127]:
try:
    sup = getsoup(alternatives[4]['details'])

    items = sup.findAll('span', class_="ingredients-item-name")

    ingredients = []
    for i in range(0, len(items)):
        ingredients.append(items[i].text.strip())

    print(ingredients)
except:
    print("smth wrong")

['½ cup uncooked brown rice', '1 cup water', '2 (16 ounce) cans black beans, rinsed and drained', '1  green bell pepper, halved and seeded', '1  onion, quartered', '½ cup sliced mushrooms', '6 cloves garlic, peeled', '¾ cup shredded mozzarella cheese', '2  eggs', '1 tablespoon chili powder', '1 tablespoon ground cumin', '1 tablespoon garlic salt', '1 teaspoon hot sauce', '½ cup dry bread crumbs, or as needed']


In [125]:
alternatives

[{'name': 'All-Star Veggie Burger',
  'img': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F747842.jpg',
  'details': 'https://www.allrecipes.com/recipe/81532/all-star-veggie-burger/'},
 {'name': 'Black Bean Quinoa Veggie Burgers',
  'img': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F3121000.jpg',
  'details': 'https://www.allrecipes.com/recipe/245799/black-bean-quinoa-veggie-burgers/'},
 {'name': 'Best Ever Veggie Burgers',
  'img': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fimages.media-allrecipes.com%2Fuserphotos%2F8207147.jpg',
  'details': 'https://www.allrecipes.com/recipe/233618/best-ever-veggie-burgers/'},
 {'name': 'Close-up on burger with cheese and lettuce',
  'img': 'https://imagesvc.meredithcorp.io/v3/mm/image?url=https%3A%2F%2Fstatic.onecms.io%2Fwp-content%2Fuploads%2Fsites%2F43%2F2020%2F09%2F24%2FAllRecipes_Veggie_Burger_0001-200

In [128]:
ingredients

['½ cup uncooked brown rice',
 '1 cup water',
 '2 (16 ounce) cans black beans, rinsed and drained',
 '1  green bell pepper, halved and seeded',
 '1  onion, quartered',
 '½ cup sliced mushrooms',
 '6 cloves garlic, peeled',
 '¾ cup shredded mozzarella cheese',
 '2  eggs',
 '1 tablespoon chili powder',
 '1 tablespoon ground cumin',
 '1 tablespoon garlic salt',
 '1 teaspoon hot sauce',
 '½ cup dry bread crumbs, or as needed']

In [129]:
def shop(query):
    addr = "http://www.bing.com/shop?q=%s" % (urllib.parse.quote_plus(query))

    soup = getsoup(addr).find('div',id='b_content')

    
    

    return results



In [146]:
addr = "http://www.bing.com/shop?q=%s" % (urllib.parse.quote_plus(ingredients[1]))

shop = getsoup(addr)
prices = shop.findAll('div', class_="pd-price br-standardPrice promoted")
if prices ==[]:
    prices = shop.findAll('span', class_='br-focusPrice')

low = 10000
for p in prices:
    cur = float(p.text[1:])
    if cur < low :
        low = cur
print(low) 
    

0.99


In [147]:
ingredients

['½ cup uncooked brown rice',
 '1 cup water',
 '2 (16 ounce) cans black beans, rinsed and drained',
 '1  green bell pepper, halved and seeded',
 '1  onion, quartered',
 '½ cup sliced mushrooms',
 '6 cloves garlic, peeled',
 '¾ cup shredded mozzarella cheese',
 '2  eggs',
 '1 tablespoon chili powder',
 '1 tablespoon ground cumin',
 '1 tablespoon garlic salt',
 '1 teaspoon hot sauce',
 '½ cup dry bread crumbs, or as needed']

In [182]:
def parseingredient(ing):
    ing.isalnum()
    ing = ''.join([i for i in ing if not i.isdigit()])
    arr = ing.split()
    nogo = ['and', 'unce', 'ablespoon', 'easpoon', 'cup', 'or', 'as' ]
    fin = ""
    for a in arr:
        ok = True
        for n in nogo:
            if n in a:
                ok = False
        if ok and "ed" not in a[len(a)-2:]:
            fin = fin+a+" "
    return re.sub('[^A-Za-z0-9 ]+', '', fin).replace(" ", "+")

parsed = []
for p in ingredients:
    parsed.append(parseingredient(p))

In [190]:
addr = "https://www.nutritionvalue.org/search.php?food_query="+parsed[4]
soup = getsoup(addr)

options = soup.find_all("a", class_="table_item_name")
soup = getsoup("https://www.nutritionvalue.org"+options[0]['href'])
calories = soup.find('td', rowspan='2', class_='right').text

In [191]:
calories

'40'